# Translation with a Sequence to Sequence Network and Attention

- http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

- フランス語を英語に翻訳するタスク
- http://www.manythings.org/anki/

In [45]:
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

use_cuda = torch.cuda.is_available()

## Loading data files

- 各単語を one-hot-encoding で表す

In [2]:
SOS_token = 0  # Start of Sentence
EOS_token = 1  # End of Sentence

class Lang:
    
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2
    
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# TEST
english = Lang('english')
english.addSentence('Hello world')
english.addSentence('I am a student')
english.addSentence('I am a dog')

print(english.word2index)
print(english.index2word)
print(english.word2count)

{'Hello': 2, 'world': 3, 'I': 4, 'am': 5, 'a': 6, 'student': 7, 'dog': 8}
{0: 'SOS', 1: 'EOS', 2: 'Hello', 3: 'world', 4: 'I', 5: 'am', 6: 'a', 7: 'student', 8: 'dog'}
{'Hello': 1, 'world': 1, 'I': 2, 'am': 2, 'a': 2, 'student': 1, 'dog': 1}


- UnicodeをAsciiに変換
- .!?以外の句読点を削除
- アルファベット以外の文字は空白に
- 小文字に変換

In [4]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r'([.!?])', r' \1', s)
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
    return s

In [5]:
normalizeString('ああ Ślusàrski. hello, world!')

' slusarski . hello world !'

- この段階ではLangオブジェクトを作るだけで空のまま

In [6]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [7]:
input_lang, output_lang, pairs = readLangs('eng', 'fra')

Reading lines...


In [8]:
len(pairs)

135842

In [9]:
pairs[:10]

[['go .', 'va !'],
 ['run !', 'cours !'],
 ['run !', 'courez !'],
 ['wow !', 'ca alors !'],
 ['fire !', 'au feu !'],
 ['help !', 'a l aide !'],
 ['jump .', 'saute .'],
 ['stop !', 'ca suffit !'],
 ['stop !', 'stop !'],
 ['stop !', 'arrete toi !']]

- 長さが10単語以内で
- I am, He is ... という文章に絞り込む

In [10]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

# 対象とする文章か判定
# p[0]にfra、p[1]にengが来ることが前提
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

# 対象とする文章に絞り込む
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [11]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

- eng-fra.txtというファイルしかないのでlang1=eng、lang2=fraとするしかない
- fra => engとしたいのでreverse=Trueで登録しておく

In [12]:
input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

Reading lines...
Read 135842 sentence pairs
Trimmed to 10853 sentence pairs
Counting words...
Counted words:
fra 4489
eng 2925


In [13]:
print(random.choice(pairs))

['je ne suis pas ton ami .', 'i m no friend of yours .']


## The Seq2Seq Model

- `forward()` は文の単語シーケンスの1つの単語のみを入れる実装になっている
- 文は外側でforループを回す
- 文を単語のシーケンスとしてforwardにまとめて入れる実装にしてもよい？

In [14]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = Variable(torch.LongTensor(indexes).view(-1, 1))
    if use_cuda:
        return result.cuda()
    else:
        return result


def variablesFromPair(pair):
    input_variable = variableFromSentence(input_lang, pair[0])
    target_variable = variableFromSentence(output_lang, pair[1])
    return (input_variable, target_variable)

In [15]:
class EncoderRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
    
    def forward(self, input, prev_hidden):
        # inputのシーケンス長は1なので文をまるごとではなく単語を1つだけ入れる
        # (seq_len, batch, input_size)
#         print('input:', input.size())
#         print('prev_hidden:', prev_hidden.size())
        embedded = self.embedding(input).view(1, 1, -1)
#         print('embedded:', embedded.size())
        output = embedded
        # outputはシーケンスの各要素を入れたときの出力がすべて保存される
        # hiddenはシーケンスの最後の要素を入れたあとの状態
        output, hidden = self.gru(output, prev_hidden)
#         print('output:', output.size())
#         print('hidden:', hidden.size())
        return output, hidden
    
    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

In [33]:
class DecoderRNN(nn.Module):
    
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, input, prev_hidden):
#         print('input:', input.size())
#         print('prev_hidden:', prev_hidden.size())

        # (seq_len, batch, input_size)
        output = self.embedding(input).view(1, 1, -1)
#         print('embedded:', output.size())
        output = F.relu(output)
        output, hidden = self.gru(output, prev_hidden)
#         print('output:', output.size())
#         print('hidden:', hidden.size())
        # シーケンス長は1なので[0]のみ
        output = self.softmax(self.out(output[0]))
#         print('softmax:', output.size())
        return output, hidden
    
    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

In [17]:
# ENCODER TEST
hidden_size = 256
encoder = EncoderRNN(input_lang.n_words, hidden_size)

# データ
train_pair = variablesFromPair(random.choice(pairs))
input = train_pair[0]   # french word_id list
target = train_pair[1]  # english word_id list

input_length = input.size()[0]
target_length = target.size()[0]

print(input_length, target_length)

# 初期状態
encoder_hidden = encoder.initHidden()
print(encoder_hidden.size())

# 系列の要素を1つずつ入力する
# 隠れ状態は上書きしていく

# encoderの出力はAttention Decoderで使うので保存しておく
encoder_outputs = Variable(torch.zeros(MAX_LENGTH, encoder.hidden_size))

for i in range(input_length):
    encoder_output, encoder_hidden = encoder(input[i], encoder_hidden)
    encoder_outputs[i] = encoder_output[0][0]

5 5
torch.Size([1, 1, 256])


In [18]:
# DECODER TEST
decoder = DecoderRNN(hidden_size, output_lang.n_words)

# Decoderの入力初期値
decoder_input = Variable(torch.LongTensor([SOS_token]))
# Decoderの隠れ状態初期値はEncoderの隠れ状態
decoder_hidden = encoder_hidden

for i in range(target_length):
    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
    # 次の正解を入れる
    decoder_input = target[i]

input: torch.Size([1])
prev_hidden: torch.Size([1, 1, 256])
embedded: torch.Size([1, 1, 256])
output: torch.Size([1, 1, 256])
hidden: torch.Size([1, 1, 256])
softmax: torch.Size([1, 2925])
input: torch.Size([1])
prev_hidden: torch.Size([1, 1, 256])
embedded: torch.Size([1, 1, 256])
output: torch.Size([1, 1, 256])
hidden: torch.Size([1, 1, 256])
softmax: torch.Size([1, 2925])
input: torch.Size([1])
prev_hidden: torch.Size([1, 1, 256])
embedded: torch.Size([1, 1, 256])
output: torch.Size([1, 1, 256])
hidden: torch.Size([1, 1, 256])
softmax: torch.Size([1, 2925])
input: torch.Size([1])
prev_hidden: torch.Size([1, 1, 256])
embedded: torch.Size([1, 1, 256])
output: torch.Size([1, 1, 256])
hidden: torch.Size([1, 1, 256])
softmax: torch.Size([1, 2925])
input: torch.Size([1])
prev_hidden: torch.Size([1, 1, 256])
embedded: torch.Size([1, 1, 256])
output: torch.Size([1, 1, 256])
hidden: torch.Size([1, 1, 256])
softmax: torch.Size([1, 2925])


## Decoder with Attention

In [39]:
class AttnDecoderRNN(nn.Module):
    
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)

        # attetnionの重みを求める写像
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        
        # decoder入力とcontextをまとめる写像？
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, prev_hidden, encoder_outputs):
#         print('input:', input.size())
#         print('prev_hidden:', prev_hidden.size())
#         print('encoder_outputs:', encoder_outputs.size())
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
#         print('embedded:', embedded.size())

        # catするのはprev_hiddenとDecoderへの入力 embedded であっている？
        # 論文の式(6)を見るとencoder_outputsな気がする
        # atten_weights = alpha
#         print('before:', torch.cat((embedded[0], prev_hidden[0]), 1).size())
        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], prev_hidden[0]), 1)), dim=1)
 
#         print('attn_weights:', attn_weights.size())

        # encoderの出力をattentionで重み付け
        # 式(5) attn_applied = ci
        # encoder_outputs = hj
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        
#         print(attn_weights.unsqueeze(0).size())
#         print(encoder_outputs.unsqueeze(0).size())
#         print('attn_applied:', attn_applied.size())

        # embedded[0] = y_{i-1}
        # (1) decoderへの入力 と (3) attentionされたcontextをまとめる写像
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)
        
        # outputがattentionで重み付けされたcontext vector？
        # gruのhiddenの方に入れると思ってたけどoutputに入れている
        # prev_hiddenは別に入れる必要があるのでoutputに入れている
        # (1) decoderへの入力 (2) decoderのprev_hidden (3) attentionされたcontextを入れる必要ある
        # 3つは入れられないので (1) と (3) をまとめてinputしている？
        output, hidden = self.gru(output, prev_hidden)
        
#         print('output:', output.size())
#         print('hidden:', hidden.size())
        
        output = F.log_softmax(self.out(output[0]), dim=1)
#         print(output.size())
        return output, hidden, attn_weights

In [20]:
MAX_LENGTH

10

In [21]:
# ATTENTION DECODER TEST
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, max_length=MAX_LENGTH)

# Decoderの入力初期値（SOSから始める）
decoder_input = Variable(torch.LongTensor([SOS_token]))

# Decoderの隠れ状態初期値はEncoderの隠れ状態
decoder_hidden = encoder_hidden

for i in range(target_length):
    decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
    decoder_input = target[i]
    break

input: torch.Size([1])
prev_hidden: torch.Size([1, 1, 256])
encoder_outputs: torch.Size([10, 256])
embedded: torch.Size([1, 1, 256])
before: torch.Size([1, 512])
attn_weights: torch.Size([1, 10])
torch.Size([1, 1, 10])
torch.Size([1, 10, 256])
attn_applied: torch.Size([1, 1, 256])
torch.Size([1, 512])
torch.Size([1, 256])
torch.Size([1, 1, 256])
output: torch.Size([1, 1, 256])
hidden: torch.Size([1, 1, 256])
torch.Size([1, 2925])


## Training the Model

In [26]:
teacher_forcing_ratio = 0.5

def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    """1つの系列長対から学習"""
    # encoderの隠れ状態を初期化
    encoder_hidden = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    # 入力の系列長
    input_length = input_variable.size()[0]
    # 出力の系列長
    target_length = target_variable.size()[0]

    # attentionではencoderの出力が必要なので保存しておく
    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    if use_cuda:
        encoder_outputs.cuda()

    loss = 0
    
    # 系列の各要素をforループで処理していく
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0][0]  # 3D tensorではなく256次元ベクトルのみ保存

    # decoderへの最初の入力は <SOS> から始める
    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    if use_cuda:
        decoder_input.cuda()

    # encoderの最後の出力をdecoderの初期状態とする
    decoder_hidden = encoder_hidden

    # 50%の確率でDecoderに教師信号を入力し、50%の確率で1つ前に生成した出力を入力する
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # Teacher forcing: 次の入力としてターゲット（正解の教師）を入力する
        for di in range(target_length):
            # Attentionを使うのでencoder_outputsを入力とする
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_variable[di])
            # 次のdecoderへの入力として正解を入れる
            decoder_input = target_variable[di]
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_variable[di])
            
            # 次のdecoderへの入力は今回の出力を入れる
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]  # 分類された単語のインデックス
            decoder_input = Variable(torch.LongTensor([[ni]]))
            if use_cuda:
                decoder_input.cuda()
    
            # 系列終了の出力が出たらそこで終わり
            if ni == EOS_token:
                break
    
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0] / target_length

In [46]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [47]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0
    
    # optimizers
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    # training data
    # iterationの数だけランダムに (french, english) の文章ペアデータを生成する
    training_pairs = [variablesFromPair(random.choice(pairs)) for i in range(n_iters)]

    # loss
    criterion = nn.NLLLoss()
    
    # 訓練データの (french, english) の系列対を学習
    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_variable = training_pair[0]   # french word_id list
        target_variable = training_pair[1]  # english word_id list
        
        # 1つの系列対から学習
        loss = train(input_variable, target_variable,
                     encoder, decoder,
                     encoder_optimizer, decoder_optimizer,
                     criterion)
        
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%d %d%% %.4f' % (iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
        
    showPlot(plot_losses)

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1)

if use_cuda:
    encoder1 = encoder1.cuda()
    attn_decoder1 = attn_decoder1.cuda()

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)